***Rushya Puttam***

**Installing PySpark**

In [85]:
pip install pyspark

**Importing required libraries**

In [86]:
from pyspark import SparkContext, SparkConf
import sys
from operator import add

In [87]:
import itertools
from itertools import combinations

**Initiating Spark Session**

In [ ]:
sc = SparkContext("local","Project 1 Apriori Algorithm")

**Reading Input file**

In [89]:
input='/content/drive/My Drive/CS-5683/browsing.txt'

In [90]:
inputRDD=sc.textFile(input)

**Setting support to 85**

In [91]:
s=85

**Modifying the given input. Removing the trailing spaces for each line in input and converting to frozensets**

In [92]:
def modify_input(rdditem):
  rdditem=rdditem.strip()
  rdditem=rdditem.split()
  rdditem=frozenset(rdditem)
  return rdditem

In [93]:
#This gives frozenset for each line in the input as map is used
intermediate_input=inputRDD.map(lambda x:modify_input(x))

In [94]:
#This gives each item separetly in the input file as flatmap is used. Also converting to frozenset
modified_input=inputRDD.flatMap(lambda x:modify_input(x)).map(lambda x :frozenset((x,)))

**Generating singletons(singletons with their whole count and the singletons with count >=s)**

In [95]:
#Generating Candidate Singletons
candidate_singletons=modified_input.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

In [96]:
#Generating Frequent Singletons
freq_singletons=candidate_singletons.filter(lambda x:x[1]>=s)

**Recursive for loop to generate 2-freq itemsets, 3-freq itemsets and 4-freq itemsets from 1-freq itemsets**

***Broadcasted the singletons and generating combinations for every line in input if the prev freq tems are a subset of each line in input***

In [97]:
def get_freqitems(freq_items,k):
  freq_combikeys=freq_items.map(lambda x:x[0])
  freq_combi_bd=sc.broadcast(freq_combikeys.collect())
  def get_combi(rdd):
      if len(rdd)>k-1: 
        for i in freq_combi_bd.value:
          if i.issubset(rdd):
            return tuple(itertools.combinations(rdd,k))
      return ""
  next_combi=intermediate_input.flatMap(get_combi) 
  candidate_itemsets=next_combi.map(lambda x:frozenset(x)).map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)
  frequent_itemsets=candidate_itemsets.filter(lambda x:x[1]>s)
  if (k<4):
    temp=frequent_itemsets
    #print(temp.take(2))
    frequent_itemsets=get_freqitems(frequent_itemsets,k+1)
    #print(frequent_itemsets.take(2))
    return temp.union(frequent_itemsets)
  return frequent_itemsets

**Calling recursive for loop to generate 2,3,4- Freq items**

In [98]:
Freq_itemsets_234=get_freqitems(freq_singletons,2)

**Generating Association Rules**

In [99]:
#Keeping only 1,2,3 freq itemsets since 4-freq items never comes to denominator while geenrating association rules
Freq_itemsets_123=freq_singletons+Freq_itemsets_234.filter(lambda x:len(x[0])<4)

In [100]:
#Creating a dictionary for 1,2,3 Freq itemsets and broadcasting the dictionary
Freq_itemsets_123_dict = {i[0] : i[1] for i in Freq_itemsets_123.collect()}
Freq_itemsets_123_dictbd=sc.broadcast(Freq_itemsets_123_dict)

**Setting the confidence to 90%**

In [101]:
c=90

In [102]:
def assocrules_generation(rdditem):
  num = rdditem[1]
  length = len(rdditem[0])
  comb = itertools.combinations(rdditem[0],length - 1)
  lst = []
  for i in comb:
    denom = Freq_itemsets_123_dictbd.value[frozenset((i))]
    confidence = (num / denom)*100
    if confidence >= c: 
      #output format
      x1 = ''
      for j in i:
        x1 = x1 + ',' + str(j)
      x1 = x1.strip(',')
      x2 = rdditem[0] - frozenset(i)
      x2 = str([k for k in x2][0])
      lst.append( x1 + ' -> ' + x2 + ' ; Confidence = ' + str(round(confidence,2)) + '%')
  return lst

In [103]:
assoc_rules=Freq_itemsets_234.flatMap(assocrules_generation)

In [104]:
assoc_rules.saveAsTextFile('/content/drive/My Drive/CS-5683/Project1_output')